In [80]:
using BenchmarkTools

include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

In [46]:
subhalo_mass1, _ = cj.subhalo_mass_function(1e+12, 1e+8)

1.0237392587587237 83 0.15456012666655014


3.791562151047401 137 0.01692825902125903


([0.00011049404064322792, 0.0009600991340222815, 0.00015408779732678225, 0.007053206938127128, 0.0002895352005843468, 0.0002451888019551688, 0.00013939555477534627, 0.0006423219281683053, 0.0002218967241473113, 0.00011806154100600224  …  0.00014438337789451344, 0.0013532950159385886, 0.0016818156324327855, 0.0002900654566719578, 0.0009664927706517039, 0.0007744623462477481, 0.00015442512728832875, 0.00012814147197611682, 0.0004351776348480336, 0.00013038605773452823], [0.014428627977395081, 0.015813936921404093, 0.01825360078328231, 0.03303032390704175, 0.03445061520596937, 0.043027498344599735, 0.04617905150371371, 0.0486412024092715, 0.0491184808925027, 0.053489745730985305  …  4.279990811133757, 4.803248684853315, 4.848709550569765, 5.960888427565806, 5.984900261159055, 6.0987486760132015, 6.56951914425008, 7.529946311376989, 7.946505447616801, 8.533726654946804])

In [81]:
subhalo_mass = cj.subhalo_mass_function_array(1e+12, 1e+8)
#@benchmark cj.one_step_merger_tree([0.1, 0.8], [0.01, 0.01], [1e+10, 1e+10], 1e+8, cj.planck18, x->x^2)
#@benchmark cj.one_step_merger_tree(0.1, 0.01, 1e+10, 1e+8)

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 10 and 2

In [48]:
m_array = 10.0.^range(-9, stop=1, length=3000)
CMF1 = [count(x -> x > m, subhalo_mass1) for m in m_array]
CMF2 = [count(x -> x > m, subhalo_mass2) for m in m_array]
CMF3 = [count(x -> x > m, subhalo_mass3) for m in m_array]
CMF4 = [count(x -> x > m, subhalo_mass4) for m in m_array]
CMF5 = [count(x -> x > m, subhalo_mass5) for m in m_array]

plot(m_array[CMF1 .> 0], CMF1[CMF1 .> 0], xscale=:log10, yscale=:log10)
plot!(m_array[CMF2 .> 0], CMF2[CMF2 .> 0], xscale=:log10, yscale=:log10)
plot!(m_array[CMF3 .> 0], CMF3[CMF3 .> 0], xscale=:log10, yscale=:log10)
plot!(m_array[CMF4 .> 0], CMF4[CMF4 .> 0], xscale=:log10, yscale=:log10)
plot!(m_array[CMF5 .> 0], CMF5[CMF5 .> 0], xscale=:log10, yscale=:log10)

UndefVarError: UndefVarError: subhalo_mass2 not defined

In [49]:
m_array = 10.0.^range(-4, stop=1, length=3000)
CMF = [[count(x -> x > m, sub_mass) for m in m_array] for sub_mass in subhalo_mass]

for sub_mass in subhalo_mass
    CMF = [count(x -> x > m, sub_mass) for m in m_array]
    plot!(m_array[CMF .> 0], CMF[CMF .> 0], xscale=:log10, yscale=:log10)

UndefVarError: UndefVarError: subhalo_mass not defined

In [50]:
using StatsBase
using Plots

result = fit(Histogram, log10.(subhalo_mass), nbins=50)
result = 
println(size(result.edges[1]), size(result.weights))

plot(result.edges[1][2:end], log10.(result.weights), st=:stairs)

UndefVarError: UndefVarError: subhalo_mass not defined

In [51]:
function one_step_merger_tree(P::Vector{<:Real}, F::Vector{<:Real}, M1::Vector{<:Real}, Mres::Real)

    n = length(M1)
    _R = rand(Float64, n)

    M2 = zeros(n)
    array_progenitors = zeros(n, 2)

    id_tree_0 = ((P .< _R) .| (M1 .< 2 * Mres)  .&  (M1 .* (1.0 .-F) .> Mres)) 
    array_progenitors[id_tree_0, 1] =  M1[id_tree_0] .* (1.0 .- F[id_tree_0])

    id_tree_1 = ((P .> _R) .& (M1 .> 2 * Mres))

    #_random = rand(Float64, sum(id_tree_1))
    #M2[id_tree_1] = draw_mass_with_restrictions.(_random, M1[id_tree_1], Mres, itp_functionP, cosmology = cosmology)
    #array_progenitors[id_tree_1, 2] =  M2[id_tree_1]
    #id_tree_2 = (id_tree_1 .& (M1 .* (1.0 .- F) .- M2 .> Mres))
    #array_progenitors[id_tree_2, 1] =  M1[id_tree_2] .* (1.0 .- F[id_tree_2]) .- M2[id_tree_2]

    #return array_progenitors
end

@benchmark one_step_merger_tree([0.1, 0.8], [0.01, 0.01], [1e+10, 1e+10], 1e+8)

BenchmarkTools.Trial: 10000 samples with 202 evaluations.
 Range (min … max):  401.609 ns … 56.302 μs  ┊ GC (min … max):  0.00% … 98.98%
 Time  (median):     415.842 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   511.004 ns ±  2.107 μs  ┊ GC (mean ± σ):  16.46% ±  3.96%

   ▅█▁                                                          
  ▃███▅▆▇▆▄▄▃▃▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  402 ns          Histogram: frequency by time          601 ns <

 Memory estimate: 884 bytes, allocs estimate: 13.